In [ ]:
from pathlib import Path
import os
import functools
from itertools import chain, combinations

In [ ]:
fp = os.path.join(Path().absolute(), "inputs", "input12.txt")
# fp = os.path.join(Path().absolute(), "inputs", "input12_test.txt")

with open(fp, "r") as f:
    data = f.read().split("\n")[:-1]

In [ ]:
data

# Part 1

In [ ]:
for line in data:
    pattern, groups = line.split()
    groups = [int(x) for x in groups.split(",")]

In [ ]:
def powerset(s):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    return list(chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))

In [ ]:
list(powerset([1,2,3]))

In [ ]:
def is_valid(pattern_new, groups):

    hash_indices = [idx for idx, char in enumerate(pattern_new) if char == "#"]
    dividers = [i for i in range(len(hash_indices) - 1) if hash_indices[i + 1] - hash_indices[i] > 1]
    dividers = [-1] + dividers + [len(hash_indices) - 1]
    groups_found = [dividers[i + 1] - dividers[i] for i in range(len(dividers) - 1)]
    return groups == groups_found

In [ ]:
is_valid(".###.##.#...", [3, 2, 1])

In [ ]:
def get_num_arrangements(pattern, groups):

    fields_to_replace = [idx for idx, char in enumerate(pattern) if char == "?"]
    # subsets = powerset(fields_to_replace)

    num_hashes_already_filled_in = len([char for char in pattern if char == "#"])
    num_hashes_required = sum(groups)
    num_hashes_to_add = num_hashes_required - num_hashes_already_filled_in
    subsets = list(combinations(fields_to_replace, num_hashes_to_add))
    print(len(subsets))

    num_valid_arrangements = 0
    for subset in subsets:

        pattern_new = [None] * len(pattern)
        for idx, char in enumerate(pattern):
            if idx in fields_to_replace:
                if idx in subset:
                    pattern_new[idx] = "#"
                else:
                    pattern_new[idx] = "."
            else:
                pattern_new[idx] = char 

        pattern_new = "".join(pattern_new)

        if is_valid(pattern_new, groups):
            num_valid_arrangements += 1

    return num_valid_arrangements

In [ ]:
total = 0
for i, line in enumerate(data):
    print(i, line)
    pattern, groups = line.split()
    groups = [int(x) for x in groups.split(",")]
    num = get_num_arrangements(pattern, groups)
    total += num

In [ ]:
total

# Part 2

In [ ]:
@functools.lru_cache(maxsize=None)
def get_num_arrangements_recursive(pattern, groups):

    if len(groups) == 0:

        if "#" in pattern:
            return 0
        else:
            return 1
        
    if len(pattern) == 0:
        return 0
    
    if pattern[0] == "?":
        return get_num_arrangements_recursive("#" + pattern[1:], groups) + get_num_arrangements_recursive("." + pattern[1:], groups)
    
    elif pattern[0] == ".":
        return get_num_arrangements_recursive(pattern[1:], groups)
    
    elif pattern[0] == "#":
        first_group = groups[0]
        if len(pattern) < first_group:
            return 0
        elif len(pattern) > first_group and pattern[first_group] == "#":
            # Won't have a separator after filling in #s
            return 0
        elif "." in pattern[:first_group]:
            return 0
        elif len(pattern) == first_group:
            if len(groups) == 1:
                return 1
            else:
                return 0
        else:
            new_pattern = pattern[(first_group + 1):] # replacing first first_group characters with # and then a .
            new_groups = groups[1:]

            return get_num_arrangements_recursive(new_pattern, new_groups)

    else:
        raise ValueError

Using recursive approach is MUCH faster. Still get the same result as before on original data.

In [ ]:
total = 0
for i, line in enumerate(data):
    print(i, line)
    pattern, groups = line.split()
    groups = tuple(int(x) for x in groups.split(","))
    num = get_num_arrangements_recursive(pattern, groups)
    total += num

In [ ]:
total

In [ ]:
total = 0
for i, line in enumerate(data):
    print(i, line)
    pattern, groups = line.split()
    groups = tuple(int(x) for x in groups.split(","))

    pattern = "?".join([pattern] * 5)
    groups = groups * 5

    num = get_num_arrangements_recursive(pattern, groups)
    total += num

In [ ]:
total